## Lite LLM 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

#from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Access your API key
if not os.getenv("OPENAI_API_KEY"):
   raise ValueError("OPENAI_API_KEY not found in .env file")

In [2]:
from litellm import completion
import pprint
import textwrap
from pydantic import BaseModel
from typing import Optional

/ephemeral/home/xiong/miniconda3/envs/llm/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


#### call openai with formated output 

In [3]:
# Define Pydantic model for structured weather response
class WeatherResponse(BaseModel):
    location: str
    temperature: Optional[str] = None
    conditions: Optional[str] = None
    humidity: Optional[str] = None
    wind: Optional[str] = None
    summary: str

## Call 4o mini with structured output
messages=[{"role": "system", "content": "You are a helpful weather assistant. Respond with structured weather information."},
          {"role": "user", "content": "what's the weather in SF"}]

response = completion(
    messages=messages,
    model='gpt-4o-mini',
    max_completion_tokens=100,  ## equals to max output tokens
    response_format=WeatherResponse  # Use Pydantic model for structured output
)

# Parse the structured response
weather_data = response['choices'][0]['message']['content']
print("Structured Weather Response:")
print(weather_data)

Structured Weather Response:
{"location":"San Francisco, CA","temperature":"58°F","conditions":"Partly Cloudy","humidity":"75%","wind":"NW 15 mph","summary":"The weather in San Francisco is cool and partly cloudy with a light breeze."}


#### call reasoning models 

In [5]:
## call thinking models 
# Call o1-mini thinking model to get both reasoning and final answer
messages = [{"role": "user", "content": "Explain the concept of recursion in programming with a simple example.Please think carefully and provide a detailed explanation."}]

# Make the completion call to o1-mini
response = completion(
    model="anthropic/claude-3-7-sonnet-20250219",
    messages=messages,
    #reasoning_effort="low", 
    )
# Check if reasoning content exists and print accordingly
if response['choices'][0]['message'].get('reasoning_content'):
    print("Reasoning Content:")
    print(response['choices'][0]['message']['reasoning_content'])
    print("\nFinal Answer:")
    print(response.choices[0].message.content[:100])
else:
    print("Raw Response:")
    print(response.choices[0].message.content[:100])



Raw Response:
# Understanding Recursion in Programming

Recursion is a programming technique where a function call


#### Call gogole gemini 2.5 

In [8]:
response = completion(
  model="gemini/gemini-2.5-flash",
  messages=[{"role": "user", "content": "What is the capital of France?"}],
  thinking={"type": "enabled", "budget_tokens": 1024},
)

print(response.choices[0].message.content)

The capital of France is **Paris**.


#### call any other openai competable api  [netmind Error]

In [9]:
#### there is a problem with server setup 
response = completion(
            model="openai/Qwen/QwQ-32B",
            messages=messages,
            api_base="https://api.netmind.ai/inference-api/openai/v1",
            api_key=os.getenv("NETMIND_API_KEY"),
            temperature=0.2,
            max_tokens=2000)

## Other useages 
- https://docs.litellm.ai/docs/providers/gemini